# 範例一 爬取並統計行政院環保署的公開資料集使用情況

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

driverPath = "chromedriver.exe"
s = Service(executable_path=driverPath)
driver = webdriver.Chrome(service=s)

df = pd.DataFrame(columns=['name', 'download'])

for i in range(1, 4):
    driver.get(f"https://data.epa.gov.tw/en/dataset?p={i}&size=100&sort=&topic_name=air&topic_name=water")
    dataset_items = driver.find_elements(By.CSS_SELECTOR, "div.dataset-item-header")

    for item in dataset_items:
        j = item.find_element(By.CSS_SELECTOR, "div.dataset-item-title")
        k = item.find_elements(By.CSS_SELECTOR, "div.dataset-item-subinfo")[1]
        temp = pd.DataFrame({'name':[j.text], 'download':[k.text]}, index=[0])
        df = pd.concat([df, temp], ignore_index=True)

driver.quit()
df

,name,download
0,General Quality of Major Rivers,Download times 0
1,Trophic State Index of Major Reservoirs,Download times 0
2,Tap Water Quality Examination,Download times 0
3,Pollution of Major Rivers,Download times 4
4,Amount of Wastewater Generation and Discharge,Download times 0
...,...,...
266,"The conference of the ""River restoration task ...",Download times 1
267,"The conference of the ""River restoration task ...",Download times 0
268,The project on the utilization of biogas slurr...,Download times 0
269,Monthly Tap water quality sampling testing data,Download times 2


In [23]:
df['download'] = df['download'].str.split().str[-1]
df['download'] = df['download'].astype(int)
df = df.sort_values('download', ascending=False)
df

,name,download
193,Air quality monitoring daily average values (a...,3414
222,Air quality monitoring hourly values (air poll...,931
205,Air quality monitoring hourly values (air poll...,772
129,Hourly values from photochemical assessment mo...,487
227,Acid rain monitoring values,334
...,...,...
71,Location map of air quality monitoring station...,0
70,Location map of regional groundwater quality m...,0
69,National drinking water source quality protect...,0
67,Map of water environment patrol route,0


In [24]:
statistics = df['download'].describe()
statistics_df = pd.DataFrame(statistics)
statistics_df


,download
count,271.000000
mean,39.051661
std,223.111227
min,0.000000
25%,0.000000
50%,1.000000
75%,45.000000
max,3414.000000


# 範例二 讀取並分析各觀測站收集的資訊

In [25]:
import pandas as pd

# Load data
df = pd.read_csv("Daily accumulated precipitation (updated every day).csv")

# Convert the monitordate column to datetime
df['monitordate'] = pd.to_datetime(df['monitordate'])

# Extract the data within a date range
start_date = '2023-03-01'
end_date = '2023-03-31'
df = df[(df['monitordate'] >= start_date) & (df['monitordate'] <= end_date)]

# Convert the rainfall24hr column to a numeric data type
df['rainfall24hr'] = pd.to_numeric(df['rainfall24hr'], errors='coerce')

# Group by siteid and calculate the average rainfall
average_rainfall = df.groupby('siteid')['rainfall24hr'].describe()

print(average_rainfall)

        count      mean        std  min  25%  50%   75%   max
siteid                                                       
1        31.0  5.225806  11.383349  0.0  0.0  0.0  3.00  43.5
2        31.0  1.612903   4.172748  0.0  0.0  0.0  0.50  19.5
3        31.0  3.338710   7.553572  0.0  0.0  0.0  2.75  33.5
4        31.0  1.032258   3.140954  0.0  0.0  0.0  0.00  16.5
5        31.0  1.225806   4.604054  0.0  0.0  0.0  0.50  25.5
...       ...       ...        ...  ...  ...  ...   ...   ...
78       31.0  0.338710   0.850364  0.0  0.0  0.0  0.00   3.5
80       31.0  0.564516   1.249946  0.0  0.0  0.0  0.50   4.5
83       31.0  0.161290   0.723269  0.0  0.0  0.0  0.00   4.0
84       31.0  1.483871   5.411783  0.0  0.0  0.0  0.40  29.8
85       31.0  0.403226   1.630027  0.0  0.0  0.0  0.00   8.5

[78 rows x 8 columns]


# 範例三 爬取動態載入 javascript

在嘗試取得台灣各城市天氣預報時，我們會預到另一個明明見到但找不到 table 表格的情況，其原因是因為這個表格是使用網頁上的動態代碼 JavaScript 動態取得的，而當我們使用 read_html 下載原網頁時，由於未有如瀏覽器般執行 JavaScript 及動態載入此數據，所以 read_html 找不到數據表格。

我們若在表格右鍵，又看不到「本頁框」，即是這個表格不是被 iframe 包著。那麼，這時可以按 F12 打開開發者工具，選擇「網絡」，重新載入網頁後，可以看到這個網頁的所有檔案載入，包括圖片檔案等。

我們想查看的是動態載入的數據。我們可以只篩選 "XHR" 類型的數據，XHR 是 JavaScript 動態載入的數據，而這個列表中，要數 ALL_Week.html 最像樣，按下去，再選擇 "Response" 返回值，可以看到果然就是我們想拿取的數據了。

When trying to obtain weather forecasts for various cities in Taiwan, we may encounter a situation where we see a table on the web page, but cannot find it in the HTML code. This is because the table is obtained dynamically using JavaScript code on the webpage, and when we use read_html to download the original webpage, it cannot find the data table as it does not execute JavaScript and dynamically load the data like a browser does.

If we right-click on the table and do not see "This Frame" option, it means that the table is not encapsulated by an iframe. At this point, we can press F12 to open the developer tools, select "Network", and reload the webpage. We can see all the files loaded by the webpage, including image files and so on.

What we want to see is the dynamically loaded data. We can filter the data type to "XHR". XHR is the data that is dynamically loaded by JavaScript. In this list, ALL_Week.html is the most likely candidate. Click on it and select "Response" to see the data that we want to retrieve.

In [26]:
import pandas as pd

url = "https://www.cwb.gov.tw/V8/E/W/week.html" # not working

url = "https://www.cwb.gov.tw/V8/E/W/County/MOD/wf7dayNC_NCSEI/ALL_Week.html"

tables = pd.read_html(url, encoding="utf8")
tables[0]

,City,Time,04/16 Sun.,04/17 Mon.,04/18 Tue.,04/19 Wed.,04/20 Thu.,04/21 Fri.,04/22 Sat.
0,Keelung City,Day,24 - 27 75 - 81,19 - 27 66 - 81,20 - 27 68 - 81,21 - 26 70 - 79,21 - 24 70 - 75,19 - 24 66 - 75,19 - 23 66 - 73
1,Keelung City,Night,19 - 25 66 - 77,20 - 25 68 - 77,21 - 25 70 - 77,21 - 24 70 - 75,19 - 22 66 - 72,19 - 22 66 - 72,19 - 22 66 - 72
2,Taipei City,Day,27 - 31 81 - 88,20 - 31 68 - 88,21 - 31 70 - 88,21 - 30 70 - 86,22 - 27 72 - 81,21 - 26 70 - 79,20 - 26 68 - 79
3,Taipei City,Night,20 - 27 68 - 81,21 - 27 70 - 81,21 - 27 70 - 81,22 - 27 72 - 81,21 - 24 70 - 75,20 - 24 68 - 75,20 - 24 68 - 75
4,New Taipei City,Day,27 - 31 81 - 88,20 - 31 68 - 88,21 - 30 70 - 86,22 - 30 72 - 86,22 - 27 72 - 81,21 - 26 70 - 79,21 - 26 70 - 79
5,New Taipei City,Night,20 - 27 68 - 81,21 - 27 70 - 81,22 - 27 72 - 81,22 - 27 72 - 81,21 - 24 70 - 75,21 - 24 70 - 75,20 - 24 68 - 75
6,Taoyuan City,Day,26 - 31 79 - 88,19 - 30 66 - 86,20 - 29 68 - 84,21 - 30 70 - 86,22 - 27 72 - 81,21 - 26 70 - 79,20 - 26 68 - 79
7,Taoyuan City,Night,19 - 26 66 - 79,20 - 26 68 - 79,21 - 26 70 - 79,22 - 27 72 - 81,21 - 25 70 - 77,20 - 24 68 - 75,20 - 24 68 - 75
8,Hsinchu City,Day,25 - 27 77 - 81,19 - 28 66 - 82,20 - 26 68 - 79,21 - 27 70 - 81,22 - 26 72 - 79,20 - 25 68 - 77,20 - 25 68 - 77
9,Hsinchu City,Night,19 - 25 66 - 77,20 - 25 68 - 77,21 - 25 70 - 77,22 - 26 72 - 79,20 - 24 68 - 75,20 - 24 68 - 75,20 - 23 68 - 73


In [27]:
# 範例四 

In [28]:
import pandas as pd

url = "https://rate.bot.com.tw/xrt?Lang=en-US"
tables = pd.read_html(url)
df = tables[0]
df # .columns

Currency                      
                                             Currency Unnamed: 1_level_1   
0        American Dollar (USD)  American Dollar (USD)             30.055  \
1      Hong Kong Dollar (HKD)  Hong Kong Dollar (HKD)              3.725   
2            British Pound (GBP)  British Pound (GBP)              36.62   
3    Australian Dollar (AUD)  Australian Dollar (AUD)              20.05   
4        Canadian Dollar (CAD)  Canadian Dollar (CAD)               22.3   
5      Singapore Dollar (SGD)  Singapore Dollar (SGD)              22.32   
6                Swiss Franc (CHF)  Swiss Franc (CHF)              33.27   
7              Japanese Yen (JPY)  Japanese Yen (JPY)             0.2185   
8   South African Rand (ZAR)  South African Rand (...                  -   
9            Swedish Krona (SEK)  Swedish Krona (SEK)               2.56   
10  New Zealand Dollar (NZD)  New Zealand Dollar (...              18.41   
11                   Thai Baht (THB)  Thai Baht (THB)              0.762   
12       Philippine Peso (PHP)  Philippine Peso (PHP)             0.4844   
13   Indonesian Rupiah (IDR)  Indonesian Rupiah (IDR)            0.00168   
14                             Euro (EUR)  Euro (EUR)               32.7   
15                 Korean Won (KRW)  Korean Won (KRW)            0.02161   
16             Vietnam Dong (VND)  Vietnam Dong (VND)            0.00105   
17   Malaysian Ringgit (MYR)  Malaysian Ringgit (MYR)              5.878   
18                   China Yen (CNY)  China Yen (CNY)              4.337   

   Cash Rate         Unnamed: 4_level_0 Spot Rate                      
      Buying Selling             Buying   Selling Unnamed: 6_level_1   
0     30.725   30.38              30.53   Inquiry            Inquiry  \
1      3.929   3.846              3.916   Inquiry            Inquiry   
2      38.74  37.515             38.145   Inquiry            Inquiry   
3      20.83  20.265              20.61   Inquiry            Inquiry   
4      23.21   22.63              22.96   Inquiry            Inquiry   
5      23.23   22.79              23.01   Inquiry            Inquiry   
6      34.47   33.88              34.27   Inquiry            Inquiry   
7     0.2313  0.2253             0.2303   Inquiry            Inquiry   
8          -   1.639              1.729   Inquiry            Inquiry   
9       3.08    2.89               3.01   Inquiry            Inquiry   
10     19.26   18.74              19.04   Inquiry            Inquiry   
11     0.952  0.8714             0.9174   Inquiry            Inquiry   
12    0.6164       -                  -   Inquiry            Inquiry   
13   0.00238       -                  -   Inquiry            Inquiry   
14     34.04  33.215             33.815   Inquiry            Inquiry   
15   0.02551       -                  -   Inquiry            Inquiry   
16   0.00146       -                  -   Inquiry            Inquiry   
17     7.398       -                  -   Inquiry            Inquiry   
18     4.499   4.404              4.464   Inquiry            Inquiry   

   Unnamed: 7_level_0  Forward History         Cash Rate             
               Buying  Forward History Selling  Buying.1 Selling.1   
0              30.055   30.725   30.38   30.53       NaN       NaN  \
1               3.725    3.929   3.846   3.916       NaN       NaN   
2               36.62    38.74  37.515  38.145       NaN       NaN   
3               20.05    20.83  20.265   20.61       NaN       NaN   
4                22.3    23.21   22.63   22.96       NaN       NaN   
5               22.32    23.23   22.79   23.01       NaN       NaN   
6               33.27    34.47   33.88   34.27       NaN       NaN   
7              0.2185   0.2313  0.2253  0.2303       NaN       NaN   
8                   -        -   1.639   1.729       NaN       NaN   
9                2.56     3.08    2.89    3.01       NaN       NaN   
10              18.41    19.26   18.74   19.04       NaN       NaN   
11              0.762    

In [29]:
df.columns

MultiIndex([(           'Currency',            'Currency'),
            (           'Currency',  'Unnamed: 1_level_1'),
            (          'Cash Rate',              'Buying'),
            (          'Cash Rate',             'Selling'),
            ( 'Unnamed: 4_level_0',              'Buying'),
            (          'Spot Rate',             'Selling'),
            (          'Spot Rate',  'Unnamed: 6_level_1'),
            ( 'Unnamed: 7_level_0',              'Buying'),
            (            'Forward',             'Forward'),
            (            'History',             'History'),
            (            'History',             'Selling'),
            (          'Cash Rate',            'Buying.1'),
            (          'Cash Rate',           'Selling.1'),
            ('Unnamed: 13_level_0', 'Unnamed: 13_level_1'),
            (          'Spot Rate', 'Unnamed: 14_level_1'),
            (          'Spot Rate', 'Unnamed: 15_level_1'),
            ('Unnamed: 16_level_0', 'Unn

In [30]:
df = df[[df.columns[0], df.columns[2]]]
df.columns = ["Currency", "Cash Rate(Selling)"]
df

,Currency,Cash Rate(Selling)
0,American Dollar (USD) American Dollar (USD),30.725
1,Hong Kong Dollar (HKD) Hong Kong Dollar (HKD),3.929
2,British Pound (GBP) British Pound (GBP),38.74
3,Australian Dollar (AUD) Australian Dollar (AUD),20.83
4,Canadian Dollar (CAD) Canadian Dollar (CAD),23.21
5,Singapore Dollar (SGD) Singapore Dollar (SGD),23.23
6,Swiss Franc (CHF) Swiss Franc (CHF),34.47
7,Japanese Yen (JPY) Japanese Yen (JPY),0.2313
8,South African Rand (ZAR) South African Rand (...,-
9,Swedish Krona (SEK) Swedish Krona (SEK),3.08


In [31]:
df.loc[:, "Currency"] = df.loc[:, "Currency"].apply(lambda x: x.split("(")[0].strip())
df.set_index("Currency", inplace=True)
df

,Cash Rate(Selling)
Currency,
American Dollar,30.725
Hong Kong Dollar,3.929
British Pound,38.74
Australian Dollar,20.83
Canadian Dollar,23.21
Singapore Dollar,23.23
Swiss Franc,34.47
Japanese Yen,0.2313
South African Rand,-
